In [ ]:
import py_mydata_RNN as mrdata
import py_mymodel_RNN as mrmodel
import py_mypipline_RNN as mrpipl
import py_mypipline_S2S as mspipl
import torch,math,collections
import torch.nn as nn
from torch.nn import functional as F
from d2l import torch as d2l
import os,logging
# import matplotlib.pyplot as plt
LOG_FORMAT = "%(asctime)s-%(levelname)s-%(message)s"
logging.basicConfig(filename='./log/log.log', level=logging.INFO, format=LOG_FORMAT)


In [8]:
valid_lens = torch.tensor([2,3,4])
batch_size = len(valid_lens)
num_heads,num_layers = 2,2
seq_len = 5
device = 'cpu'
mat_valid = valid_lens.repeat(seq_len,num_layers,1).permute(2,1,0)
mat_arrage = torch.arange(seq_len).repeat(batch_size,num_layers,1).to(device)
mask = mat_arrage>=mat_valid
print(mat_valid)
print(mat_arrage)
print(mask)

tensor([[[2, 2, 2, 2, 2],
         [2, 2, 2, 2, 2]],

        [[3, 3, 3, 3, 3],
         [3, 3, 3, 3, 3]],

        [[4, 4, 4, 4, 4],
         [4, 4, 4, 4, 4]]])
tensor([[[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]],

        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]],

        [[0, 1, 2, 3, 4],
         [0, 1, 2, 3, 4]]])
tensor([[[False, False,  True,  True,  True],
         [False, False,  True,  True,  True]],

        [[False, False, False,  True,  True],
         [False, False, False,  True,  True]],

        [[False, False, False, False,  True],
         [False, False, False, False,  True]]])


In [7]:


# mat_valid = valid_lens.repeat(seq_len,1).T
mat_valid = valid_lens.repeat(num_heads, seq_len, seq_len, 1).permute(3,0,1,2)
print(mat_valid)
mat_arrage = torch.arange(seq_len).repeat(batch_size,num_heads, seq_len, 1).to(device)
    
# mat_arrage = torch.arange(seq_len).repeat(batch_size,1).to(device)
print(mat_arrage)
mask = mat_arrage>=mat_valid
print(mask)


tensor([[[[2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2]],

         [[2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2],
          [2, 2, 2, 2, 2]]],


        [[[3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3]],

         [[3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3],
          [3, 3, 3, 3, 3]]],


        [[[4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4]],

         [[4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4],
          [4, 4, 4, 4, 4]]]])
tensor([[[[0, 1, 2, 3, 4],
          [0, 1, 2, 3, 4],
          [0, 1, 2, 3, 4],
          [0, 1, 2, 3, 4],
          [0, 1, 2, 3, 4]],

         [[0, 1, 2, 3, 4],
         

In [9]:
mask = torch.triu(torch.ones((seq_len, seq_len)), diagonal=1).bool().to(device)
mask = mask.repeat(batch_size, num_heads, 1, 1)
print(mask)


tensor([[[[False,  True,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False, False,  True],
          [False, False, False, False, False]],

         [[False,  True,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False, False,  True],
          [False, False, False, False, False]]],


        [[[False,  True,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False, False,  True],
          [False, False, False, False, False]],

         [[False,  True,  True,  True,  True],
          [False, False,  True,  True,  True],
          [False, False, False,  True,  True],
          [False, False, False, False,  True],
          [False, False, False, False, False]]],


        [[[False,  True,  True,  True,  True],
 

In [15]:
a=torch.randn((2,3,4,5))
print(a.dim())
print(a.unsqueeze(0).shape)
b=torch.randn(())
print(b.dim())
print(b)
print(a.masked_fill(mask, -1e6))



4
torch.Size([1, 2, 3, 4, 5])
0
tensor(-0.7141)


## seq2seq attention 机制中实现attention 函数

In [ ]:
# # 针对seq2seq的数据结构，实现注意力机制

# # input arg: X, size = [batch_size, seq_len]
# # input arg: en_outputs, size = [seq_len, batch_size, dim_hiddens]
# # input arg: state, size = [num_layer *  num_direction, batch_size, dim_hiddens]
# # input arg: x_vare_len, size = [batch_size]
# X = self.emb(X).permute(1, 0, 2) # emb并调整维度顺序后 [seq_len, batch_size, dim_emb]

# ### attention
# # Q = state; K = en_outputs; V = en_outputs
# # res = F.softmax(Q.permute(1,0,2).matmul(K.permute(1,2,0))/math.sqrt(K.shape[-1])).matmul(K.permute(1,0,2))
# # dim res = (batch_size, 1, dim_hiddens)
# # 主要是维度变换的问题 对于批量矩阵相乘 只要保证前几维度相同即可自动计算后续例如size=(6,5,3,4) 和 size=(6,5,4,1)矩阵相乘得到(6,5,3,1)
# atte_scores = state.permute(1,0,2).matmul(en_outputs.permute(1,2,0))
# # 计算softmax分数
# atte_scores = F.softmax(atte_scores/math.sqrt(en_outputs.shape[-1]), dim=-1)
# atten_res = atte_scores.matmul(en_outputs.permute(1,0,2)).squeeze(1)
# context = atten_res.repeat(X.shape[0], 1, 1)
# # after cat size = [seq_len, batch_size, dim_hiddens + dim_emb]
# X_and_context = torch.cat((X, context), 2)
# output, state = self.rnn(X_and_context, state)
# output = self.dense(output).permute(1, 0, 2)
# # output的形状:(batch_size, num_steps, vocab_size)
# # state的形状: (num_layers * num_direction, batch_size, num_hiddens)

### 训练seq2seq attention模型-翻译数据

In [ ]:
# model_path = './saved_models/nmt_eng2fra_s2s_atten'
# batch_size, num_steps = 64, 16
# embed_size, num_hiddens, num_layers, dropout = 128, 128, 2, 0.2
# incellplot = False
# epoch = 100
# num_examples = 10000
# device = d2l.try_gpu()
# model_path = train_s2s_attention_pipline(model_path, batch_size, num_steps,
#                            embed_size, num_hiddens, num_layers,
#                            dropout,num_epochs=epoch, num_examples=num_examples, incellplot=incellplot)
# model_path = './saved_models/nmt_eng2fra_s2s_atten_bz64_ns16_dime128_dimh128_nl2_epo200_lr0.01_1w.pkl'
# pridict_s2s_attention_pipline(model_path, batch_size, num_steps, embed_size, num_hiddens, num_layers, dropout, device, num_examples=num_examples)
# # 0.110 atten 32 32 10000
# # avg blue:0.281, test num:5000 nmt_eng2fra_s2s_atten_bz64_ns16_dime128_dimh128_nl1_epo200_lr0.01_10w.pkl

In [7]:
batch_size, num_steps, dim = 5, 4, 3
Q = torch.randn((batch_size, num_steps, dim))
K = torch.randn((batch_size, num_steps, dim))
V = torch.randn((batch_size, num_steps, dim))
atten_socre = Q.matmul(K.permute(0,2,1))/math.sqrt(Q.shape[-1])
print(atten_socre)
atten_socre[:,:,2:]=-1e6
atten_socre = F.softmax(atten_socre, dim=-1)
outputs = atten_socre.matmul(V)
print(atten_socre)
print(outputs)

tensor([[[ 0.7109, -1.8129, -0.3297,  0.3725],
         [-1.5190,  1.9149, -0.1870, -1.5384],
         [-0.2127, -0.6461, -0.3796, -0.5093],
         [ 0.7262, -0.0470,  0.6699,  1.2201]],

        [[-0.9459,  0.3858, -0.1374,  0.8444],
         [-0.3865,  0.4512, -0.7994,  1.1063],
         [-0.7378, -0.8631,  1.7015,  0.8022],
         [-0.8016, -1.3035,  2.9029, -0.4328]],

        [[ 0.6304,  1.1267, -0.5393, -3.1494],
         [-0.1664, -0.3146,  0.1162,  1.6930],
         [ 0.2385, -0.2877, -0.2676, -0.3129],
         [ 0.0702,  0.6072, -0.0227, -0.7544]],

        [[-0.0729, -0.4624,  0.1464,  0.7713],
         [ 1.1600,  0.8432, -0.3150, -1.4627],
         [ 0.4971, -0.6775,  0.4755,  0.9159],
         [-0.0514,  0.8352, -0.5222, -1.2163]],

        [[ 0.6708, -0.0872,  1.8572,  0.8870],
         [ 0.6597, -0.8832,  0.6984, -0.5014],
         [-0.3564, -0.2620, -0.1831, -0.3153],
         [-0.4962, -0.8781, -1.5900, -1.6608]]])
tensor([[[0.9258, 0.0742, 0.0000, 0.0000],
       